# QuickGuide for rtings

[프로젝트 페이지: Research-on-the-TV-market](https://github.com/xikest/research-market-tv)

## Env setting: Install Colab selenium & crome driver

In [ ]:
!pip install -U pandas openpyxl tqdm
!pip install -U requests selenium beautifulsoup4 
!pip install -U wordcloud nltk 
!pip install -U openai 
!pip install -U matplotlib seaborn
!pip install -U getmodelspec

In [3]:
from market_research.tools.installer import Installer
Installer.install_chrome_and_driver()

## Env. 세팅

In [1]:
from datetime import date
from pathlib import Path
import pandas as pd
# from google.colab import files
from market_research import Rtings
from market_research.tools import FileManager

### 분석 폴더
- `input_data`: 검색할 데이터 폴더
- `results`: 검색된 데이터 폴더

In [2]:
intput_folder = Path("input_urls")  # 폴더 이름을 지정

# 폴더가 존재하지 않으면 폴더 생성
if not intput_folder.exists():
    intput_folder.mkdir(parents=True)

output_comments_folder = Path('results/comments')  
if not output_comments_folder.exists():
  output_comments_folder.mkdir(parents=True, exist_ok=True)
    
output_scores_folder = Path('results/scores')  
if not output_scores_folder.exists():
  output_scores_folder.mkdir(parents=True, exist_ok=True)

## 준비

In [3]:
webdriver_path = "/content/chromedriver/chromedriver"
browser_path = "/content/chrome/chrome"
enable_headless = True

- 파일 명은 `rtings_url.xlsx`으로 하고, column 명은 `urls`으로 하여 `input_data`폴더에 넣어주세요

In [5]:
# 분석할 엑셀 파일이 있는 폴더 경로
file_list = intput_folder.glob('*')
excel_files = [file for file in file_list if file.suffix in {'.xlsx', '.xls'}]
urls=[]
for excel_file in excel_files:
    df = pd.read_excel(excel_file)
    urls.extend(df["urls"])

## 실행

In [ ]:
for url in urls:
    maker = url.split("/")[-2]
    model = url.split("/")[-1]


    rtings = Rtings(webdriver_path = webdriver_path, browser_path=browser_path, enable_headless=enable_headless)

    # 저장할 데이터 경로
    file_name = f"{maker}_{model}_rtings_comments_{date.today().strftime('%Y-%m-%d')}.xlsx"
    comments_file_name= output_comments_folder/file_name
    comments_df = rtings.get_commetns(url, format_df=True)
    comments_df.to_excel(f"{comments_file_name}", index=False, sheet_name='comments')

    # 저장할 데이터 경로
    file_name = f"{maker}_{model}_rtings_scores_{date.today().strftime('%Y-%m-%d')}.xlsx"
    scores_file_name = output_scores_folder/file_name
    score_df= rtings.get_score(url,format_df=True)
    score_df.to_excel(f"{scores_file_name}", index=False, sheet_name='scores')

- colab에서 파일 받기

In [ ]:
!zip -r /content/results_rtings.zip /content/results/
files.download('/content/results_rtings.zip')

---